# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.widgets import RunDetails

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'hd-experiment'

experiment=Experiment(ws, experiment_name)

In [3]:
compute_target = ComputeTarget(workspace=ws, name='cap-compute')

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

The model (see `train.py`) is a logistic regression model, with specified hyperparameters for l1, l2 and max iteration values. `C` specifies regularization weight (for l1 and l2), `l1_ratio` specifies the proportion of regularization weight allocated to l1, compared to l2. `max_iters` specifies the maximum number of iterations.

The Bandit stopping policy was set to make sure that the iterations don't drift too far into potential overfitting, and waste time, by stopping after a delay from the best iteration.

In [4]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.2, delay_evaluation=3)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--C": uniform(0.1, 1.0),
    "--max_iter": choice(10, 25, 35, 50),
    '--l1_ratio': uniform(0, 1),
})

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory=".", compute_target=compute_target, entry_script="train.py")

hyperdrive_run_config = HyperDriveConfig(
    hyperparameter_sampling=param_sampling,
    primary_metric_name="Accuracy",
    estimator=estimator, policy=early_termination_policy,
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=16,
    max_concurrent_runs=5
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [5]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
# get best run
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()

#get best metrics
best_run_metrics = best_hyperdrive_run.get_metrics()
print(best_run_metrics)

print('Best HyperDrive RunID: ', best_hyperdrive_run.id)

In [ ]:
#TODO: Save the best model
model = best_run.register_model(model_name='keras-model', model_path='outputs/model')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service